In [10]:
import acquire as a
import pandas as pd
import prepare

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/natasharivers/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/natasharivers/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#process only one repo
test= a.process_repo('dsfsi/covid19za')

In [3]:
#take a look
test

{'repo': 'dsfsi/covid19za',
 'language': 'Jupyter Notebook',
 'readme_contents': "# Coronavirus COVID-19 (2019-nCoV) Data Repository for South Africa\n\n[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3819126.svg)](https://doi.org/10.5281/zenodo.3819126) [![dsJournal](https://img.shields.io/badge/DSJournal-10.5334-B31B1B.svg)](https://doi.org/10.5334/dsj-2020-019)\n\nCoronavirus COVID-19 (2019-nCoV) Data Repository for South Africa created, maintained and hosted by [Data Science for Social Impact research group](https://dsfsi.github.io/), led by Dr. Vukosi Marivate, at the University of Pretoria.\n\n**Disclaimer:** We have worked to keep the data as accurate as possible. We collate the COVID 19 reporting data from NICD and DoH. We only update that data once there is an official report or statement. For the other data, we work to keep the data as accurate as possible. If you find errors. Make a pull request.\n\n*If you use this repo for any research/development/innovation, please co

In [4]:
scrape_data = a.scrape_github_data()

In [7]:
df = pd.DataFrame(scrape_data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             100 non-null    object
 1   language         89 non-null     object
 2   readme_contents  100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


# Prepare

In [11]:
df = df.rename({'readme_contents':'original'},axis=1)
df['clean'] = df.original.apply(prepare.basic_clean)
df['lemmatized'] = df.clean.apply(prepare.lemmatize)
df['stopwords_removed'] = df.lemmatized.apply(prepare.remove_stopwords)
df.head()

,repo,language,original,clean,lemmatized,stopwords_removed
0,nytimes/covid-19-data,None,# Coronavirus (Covid-19) Data in the United St...,coronavirus covid19 data in the united states...,coronavirus covid19 data in the united state t...,coronavirus covid19 data united state primary ...
1,covid19india/covid19india-react,JavaScript,"<p align=""center"">\n<img src=""https://reposito...",p aligncenter\nimg srchttpsrepositoryimagesgit...,p aligncenter img srchttpsrepositoryimagesgith...,p aligncenter img srchttpsrepositoryimagesgith...
2,owid/covid-19-data,Python,# COVID-19 Dataset by _Our World in Data_ \n[!...,covid19 dataset by our world in data \nwebsit...,covid19 dataset by our world in data website s...,covid19 dataset world data website shieldsioht...
3,SeanNaren/CORD-19-ANN,Python,# CORD-19-ANN\n\n![cord_website](imgs/cord_ann...,cord19ann\n\ncordwebsiteimgscordannexamplegif...,cord19ann cordwebsiteimgscordannexamplegif ope...,cord19ann cordwebsiteimgscordannexamplegif ope...
4,geohot/corona,Python,# Reverse engineering the coronavirus (SARS-Co...,reverse engineering the coronavirus sarscov2\...,reverse engineering the coronavirus sarscov2 s...,reverse engineering coronavirus sarscov2 start...


# Explore